In [5]:
import pandas as pd 
import geopandas as gpd 

import geopandas as gpd
import glob 
import pandas as pd
import os
import concurrent.futures
from osgeo import ogr
import pandas as pd
import os
import shutil 
import tempfile
import threading


import sys 
sys.path.append('/Users/gracecolverd/New_dataset')
from src.buildings import get_bounding_boxes , calculate_bounding_boxes, generate_new_filename

from src.pre_process_buildings import create_height_bucket_col , creat_age_buckets


In [2]:



# # Ensure thread_local is defined at the global level
# thread_local = threading.local()


# def get_thread_temp_file(log_file):
#     if not hasattr(thread_local, 'temp_file'):
#         temp_dir = os.path.dirname(log_file)
#         temp_file = tempfile.NamedTemporaryFile(delete=False, mode='w+', suffix='.csv', prefix='temp_log_', dir=temp_dir)
#         thread_local.temp_file = temp_file.name
#         thread_local.temp_file_first_write = True
#     return thread_local.temp_file, thread_local.temp_file_first_write


# def process_batch(bbox_list, output_directory, log_file="proc/processing_log.csv"):
#     print('starting batch')
#     results = []
#     for bbox in bbox_list:
#         filepath, bb, status, length = process_bbox(bbox, output_directory)  # Ensure this is the correct return format
#         results.append((filepath, bb, status, length))
#     df = pd.DataFrame(results, columns=['filepath', 'bbox', 'status', 'len'])
#     print('results done, starting save')
#     # Get the temp file path and whether it's the first write operation
#     temp_file_path, is_first_write = get_thread_temp_file(log_file)
#     print('temp file path is ' , temp_file_path )
#     # Open the file with 'a' mode to append and ensure headers are written correctly
#     with open(temp_file_path, 'a') as f:
#         df.to_csv(f, header=is_first_write, index=False)
    
#     # Update the flag to indicate the header should not be written next time
#     if is_first_write:
#         thread_local.temp_file_first_write = False

#     print('temp file saved for batch')



# def run_batching(bounding_boxes, output_directory, log_file="proc/processing_log.csv"):
#     batch_size = 5
#     with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
#         # Store futures if you need to wait for them or check for exceptions
#         futures = []
#         for i in range(0, len(bounding_boxes), batch_size):
#             batch = bounding_boxes[i:i+batch_size]
#             future = executor.submit(process_batch, batch, output_directory, log_file)
#             futures.append(future)
#         # Optionally wait for all futures if needed
#         concurrent.futures.wait(futures)




# def merge_temp_logs_to_main(log_file="proc/processing_log.csv"):
#     """
#     Merge all temporary log files created by threads into the main log file.

#     Parameters:
#     - log_file: str, the path to the main log file.
#     """
#     # Assume temp files are in the same directory as the main log file
#     temp_files = [f for f in os.listdir(os.path.dirname(log_file)) if f.startswith('temp_log_') and f.endswith('.csv')]
#     print('num of temp files found ', len(temp_files) ) 
#     # Create or append to the main log file
#     if len(temp_files) != 0:
#         for temp_file in temp_files:
#             temp_file_path = os.path.join(os.path.dirname(log_file), temp_file)
#             df_temp = pd.read_csv(temp_file_path )
            
#             if os.path.exists(log_file):
#                 df_temp.to_csv(log_file, mode='a', header=False, index=False)
#             else:
#                 df_temp.to_csv(log_file, mode='w', header=True, index=False)
#     print('fn complete')
            
    

# def cleanup_temp_files(temp_file_prefix="temp_log_", temp_dir="proc"):
    
#     for filename in os.listdir(temp_dir):
#         if filename.startswith(temp_file_prefix) and filename.endswith('.csv'):
#             os.remove(os.path.join(temp_dir, filename))
#             print(f"Deleted temporary file: {filename}") 
    


# def generate_bbox_list(bounding_boxes, log= 'proc/processing_log.csv'): 
#     print('Start generate_bbox_list')
#     if os.path.exists(log):
#         log = pd.read_csv(log)
#         complete = log[log['status']=='completed']['bbox'].tolist() 
#         bbox_to_process = [bbox for bbox in bounding_boxes if str(bbox) not in complete] 
    
#     else:
#         bbox_to_process = bounding_boxes
#     print('num to process ',   len(bbox_to_process) ) 
#     return bbox_to_process 

# def cleanup_output_files(output_dir   ):
#     log = pd.read_csv('proc/processing_log.csv') 
#     if len(log) == 0:
#         print('log not found')
#         return
#     folder = glob.glob(output_dir + '/*' ) 
#     sucess_files = log[log['status']=='completed']['filepath'].tolist()   

#     not_succ = [f for f in folder if f not in sucess_files]  
#     print('files to remove: ', len(not_succ ))
#     # remove files that are not in the log
#     for f in not_succ:
#         os.remove(f) 

In [41]:

# # # def process_bbox(bbox_list , input_gpk ):
# # #     # Placeholder for aggregated statistics and weights
# # #     listdf = [] 

# # #     for bbox in bbox_list:
# # #         print('Starting ', bbox )
# # #         subset = gpd.read_file(input_gpk, bbox=bbox)
# # #         if subset.empty:
# # #             print('Empty subset')
# # #             continue
# # #         group_means = subset.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['height'].mean().reset_index()
# # #         group_counts = subset.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['height'].count().reset_index(name='count')
# # #         union = pd.merge(group_means, group_counts, on=['map_simple_use', 'premise_age', 'premise_floor_count'])
# # #         union['weighted_height'] = union['height'] * union['count'] 
# # #         # print(union)
# # #         listdf.append(union)
# # #     full_df = pd.concat(listdf)

# # #     total_counts  = full_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['count'].sum().reset_index() 
# # #     total_weighted_mean = full_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['weighted_height'].sum().reset_index()
# # #     total_weighted_mean['weighted_mean'] = total_weighted_mean['weighted_height'] / total_counts['count']
    
# # #     return total_weighted_mean 
        

    
# # import geopandas as gpd
# # import pandas as pd

# # def process_bbox(bbox_list, input_gpk):
# #     listdf = []  # List to hold DataFrame fragments

# #     for bbox in bbox_list:
# #         print('Processing bounding box:', bbox)
# #         subset = gpd.read_file(input_gpk, bbox=bbox)  # Read subset within the bounding box
# #         if subset.empty:
# #             print('Empty subset for bounding box:', bbox)
# #             continue

# #         # Group by specified columns and calculate both mean height and count in one go
# #         stats = subset.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['height'] \
# #                 .agg(mean_height='mean', count='size').reset_index()
# #         stats['weighted_height'] = stats['mean_height'] * stats['count']
# #         print(len(stats))
        
# #         listdf.append(stats) 
# #         print('len listdf ', len(listdf) ) # Append the calculated stats to the list

# #     # Concatenate all DataFrames in the list into one
# #     full_df = pd.concat(listdf)

# #     # Calculate total counts and sum of weighted heights for each group across all subsets
# #     total_stats = full_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count']).agg(
# #         total_count=('count', 'sum'),
# #         sum_weighted_height=('weighted_height', 'sum')).reset_index()
    
# #     # Calculate weighted mean height for each group
# #     total_stats['weighted_mean'] = total_stats['sum_weighted_height'] / total_stats['total_count']

# #     # Cleanup before returning
# #     total_stats = total_stats.drop(columns=['total_count', 'sum_weighted_height'])

# #     total_stats = total_stats.rename(columns={'weighted_mean': 'global_average_height'})
# #     total_stats.to_csv('global_average_heights.csv')
# #     print('File saved')
    
# #     return total_stats




    
# import geopandas as gpd
# import pandas as pd

# # def process_bbox(bbox_list, input_gpk):
# #     # Initialize an empty DataFrame for storing intermediate results
# #     intermediate_dic = {}

# #     for bbox in bbox_list:
# #         print('Processing bounding box:', bbox)
# #         subset = gpd.read_file(input_gpk, bbox=bbox)  # Read subset within the bounding box
# #         if subset.empty:
# #             print('Empty subset for bounding box:', bbox)
# #             continue

# #         # Calculate both mean height and count in one go and add weighted_height column
# #         stats = subset.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['height'] \
# #                 .agg(mean_height='mean', count='size').reset_index()
# #         stats['weighted_height'] = stats['mean_height'] * stats['count']
# #         print(stats)
# #         # Append the calculated stats directly to the intermediate DataFrame
# #         intermediate_dic[bbox] = stats
    
# #     intermediate_df = pd.DataFrame(intermediate_dic)
# #     print(intermediate_df)
# #     # Now, intermediate_df contains all data from all bounding boxes
# #     # Calculate total counts and sum of weighted heights for each group across all subsets
# #     total_stats = intermediate_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count']).agg(
# #         total_count=('count', 'sum'),
# #         sum_weighted_height=('weighted_height', 'sum')).reset_index()
    
# #     # Calculate weighted mean height for each group
# #     total_stats['weighted_mean'] = total_stats['sum_weighted_height'] / total_stats['total_count']

# #     # Cleanup before returning
# #     total_stats = total_stats.drop(columns=['total_count', 'sum_weighted_height'])
# #     total_stats = total_stats.rename(columns={'weighted_mean': 'global_average_height'})

# #     # Save the result to CSV
# #     total_stats.to_csv('global_average_heights.csv', index=False)
# #     print('File saved')
    
# #     return total_stats



# import geopandas as gpd
# import pandas as pd

# def process_bbox(bbox_list, input_gpk):
#     # Initialize a dictionary to store intermediate results
#     intermediate_dict = {'map_simple_use': [], 'premise_age': [], 'premise_floor_count': [], 'mean_height': [], 'count': [], 'weighted_height': []}

#     for bbox in bbox_list:
#         print('Processing bounding box:', bbox)
#         subset = gpd.read_file(input_gpk, bbox=bbox)
#         if subset.empty:
#             print('Empty subset for bounding box:', bbox)
#             continue

#         # Calculate both mean height and count in one go and add weighted_height calculation
#         stats = subset.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['height'].agg(mean_height='mean', count='size').reset_index()
#         stats['weighted_height'] = stats['mean_height'] * stats['count']
        
#         # Store the results in the dictionary
#         for col in intermediate_dict.keys():
#             intermediate_dict[col].extend(stats[col].tolist())

#     # Convert the dictionary to a DataFrame
#     intermediate_df = pd.DataFrame(intermediate_dict)

#     # Calculate total counts and sum of weighted heights for each group across all subsets
#     total_stats = intermediate_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count']).agg(
#         total_count=('count', 'sum'),
#         sum_weighted_height=('weighted_height', 'sum')).reset_index()
    
#     # Calculate weighted mean height for each group
#     total_stats['weighted_mean'] = total_stats['sum_weighted_height'] / total_stats['total_count']

#     # Cleanup before returning
#     total_stats = total_stats.drop(columns=['total_count', 'sum_weighted_height'])
#     total_stats = total_stats.rename(columns={'weighted_mean': 'global_average_height'})

#     # Save the result to CSV
#     total_stats.to_csv('global_average_heights.csv', index=False)
#     print('File saved')
    
#     return total_stats



In [4]:

def test_medheights(bbox_list , input_gpk ):
    # Placeholder for aggregated statistics and weights

    max_file = []
    for bbox in bbox_list:
        print('Starting ', bbox )
        subset = gpd.read_file(input_gpk, bbox=bbox)
        if subset.empty:
            print('Empty subset')
            continue
        max_file.append(subset)
    max_ds = pd.concat(max_file)
    return max_ds

In [2]:
year = 2022
input_gpk = '/Volumes/T9/Data_downloads/Versik_building_data/2024_03_22_updated_data/UKBuildings_Edition_15_new_format_upn.gpkg' 


ds = ogr.Open(input_gpk)
layer = ds.GetLayer()
extent = layer.GetExtent()
bounding_boxes = calculate_bounding_boxes(extent, chunk_height=100000, chunk_width=100000)
len(bounding_boxes)


91

In [43]:
test = bounding_boxes[0:10]

medh = process_bbox(test, input_gpk) 

medh

Processing bounding box: (9252.36, 6435.25920425788, 109252.36, 106435.25920425788)
Processing bounding box: (9252.36, 106435.25920425788, 109252.36, 206435.25920425786)
Empty subset for bounding box: (9252.36, 106435.25920425788, 109252.36, 206435.25920425786)
Processing bounding box: (9252.36, 206435.25920425786, 109252.36, 306435.25920425786)
Empty subset for bounding box: (9252.36, 206435.25920425786, 109252.36, 306435.25920425786)
Processing bounding box: (9252.36, 306435.25920425786, 109252.36, 406435.25920425786)
Empty subset for bounding box: (9252.36, 306435.25920425786, 109252.36, 406435.25920425786)
Processing bounding box: (9252.36, 406435.25920425786, 109252.36, 506435.25920425786)
Empty subset for bounding box: (9252.36, 406435.25920425786, 109252.36, 506435.25920425786)
Processing bounding box: (9252.36, 506435.25920425786, 109252.36, 606435.2592042579)
Empty subset for bounding box: (9252.36, 506435.25920425786, 109252.36, 606435.2592042579)
Processing bounding box: (92

,map_simple_use,premise_age,premise_floor_count,global_average_height
0,Mixed Use,1870-1918,2,7.633334
1,Mixed Use,1870-1918,4,13.800000
2,Mixed Use,1919-1944,2,8.200000
3,Mixed Use,1945-1959,2,8.500000
4,Mixed Use,1960-1979,1,5.600000
5,Mixed Use,1960-1979,2,8.533334
6,Mixed Use,1960-1979,3,11.000000
7,Mixed Use,1990-1999,2,9.400000
8,Mixed Use,Pre 1837,2,9.600000
9,Mixed Use,Pre 1837,3,10.500000


In [44]:
full_h = process_bbox(bounding_boxes, input_gpk) 

Processing bounding box: (9252.36, 6435.25920425788, 109252.36, 106435.25920425788)
Processing bounding box: (9252.36, 106435.25920425788, 109252.36, 206435.25920425786)
Empty subset for bounding box: (9252.36, 106435.25920425788, 109252.36, 206435.25920425786)
Processing bounding box: (9252.36, 206435.25920425786, 109252.36, 306435.25920425786)
Empty subset for bounding box: (9252.36, 206435.25920425786, 109252.36, 306435.25920425786)
Processing bounding box: (9252.36, 306435.25920425786, 109252.36, 406435.25920425786)
Empty subset for bounding box: (9252.36, 306435.25920425786, 109252.36, 406435.25920425786)
Processing bounding box: (9252.36, 406435.25920425786, 109252.36, 506435.25920425786)
Empty subset for bounding box: (9252.36, 406435.25920425786, 109252.36, 506435.25920425786)
Processing bounding box: (9252.36, 506435.25920425786, 109252.36, 606435.2592042579)
Empty subset for bounding box: (9252.36, 506435.25920425786, 109252.36, 606435.2592042579)
Processing bounding box: (92

: 

In [ ]:
full_h.to_csv('temp.csv')

In [ ]:
def process_bbox_age(bbox_list, input_gpk):
    listdf = []  # List to hold DataFrame fragments

    for bbox in bbox_list:
        print('Processing bounding box:', bbox)
        subset = gpd.read_file(input_gpk, bbox=bbox)  # Read subset within the bounding box
        if subset.empty:
            print('Empty subset for bounding box:', bbox)
            continue

        # Group by specified columns and calculate both mean height and count in one go
        stats = subset.groupby(['map_simple_use', 'premise_age_bucketed', 'premise_floor_count'])['height'] \
                .agg(mean_height='mean', count='size').reset_index()
        stats['weighted_height'] = stats['mean_height'] * stats['count']
        
        listdf.append(stats)  # Append the calculated stats to the list

    # Concatenate all DataFrames in the list into one
    full_df = pd.concat(listdf)

    # Calculate total counts and sum of weighted heights for each group across all subsets
    total_stats = full_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count']).agg(
        total_count=('count', 'sum'),
        sum_weighted_height=('weighted_height', 'sum')).reset_index()
    
    # Calculate weighted mean height for each group
    total_stats['weighted_mean'] = total_stats['sum_weighted_height'] / total_stats['total_count']

    # Cleanup before returning
    total_stats = total_stats.drop(columns=['total_count', 'sum_weighted_height'])

    total_stats = total_stats.rename(columns={'weighted_mean': 'global_average_height'})
    total_stats.to_csv('global_average_heights.csv')
    print('File saved')
    
    return total_stats


In [4]:
from src.pre_process_buildings import pre_process_buildings 

In [ ]:
# for average floor heights 

def process_bbox_floor_heigh(bbox_list, input_gpk):
    listdf = []  # List to hold DataFrame fragments

    for bbox in bbox_list:
        print('Processing bounding box:', bbox)
        subset = gpd.read_file(input_gpk, bbox=bbox)  # Read subset within the bounding box
        if subset.empty:
            print('Empty subset for bounding box:', bbox)
            continue
        subset = creat_age_buckets(subset )

        # Group by specified columns and calculate both mean height and count in one go
        stats = subset.groupby(['map_simple_use', 'premise_age_bucketed', 'premise_floor_count'])['height'] \
                .agg(mean_height='mean', count='size').reset_index()
        stats['weighted_height'] = stats['mean_height'] * stats['count']
        
        listdf.append(stats)  # Append the calculated stats to the list

    # Concatenate all DataFrames in the list into one
    full_df = pd.concat(listdf)

    # Calculate total counts and sum of weighted heights for each group across all subsets
    total_stats = full_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count']).agg(
        total_count=('count', 'sum'),
        sum_weighted_height=('weighted_height', 'sum')).reset_index()
    
    # Calculate weighted mean height for each group
    total_stats['weighted_mean'] = total_stats['sum_weighted_height'] / total_stats['total_count']

    # Cleanup before returning
    total_stats = total_stats.drop(columns=['total_count', 'sum_weighted_height'])

    total_stats = total_stats.rename(columns={'weighted_mean': 'global_average_height'})
    total_stats.to_csv('global_average_heights_bucket.csv')
    print('File saved')
    
    return total_stats

ModuleNotFoundError: No module named 'src'

In [ ]:
# for average floor heights 

def process_bbox_floor_count(bbox_list, input_gpk):
    listdf = []  # List to hold DataFrame fragments

    for bbox in bbox_list:
        print('Processing bounding box:', bbox)
        subset = gpd.read_file(input_gpk, bbox=bbox)  # Read subset within the bounding box
        if subset.empty:
            print('Empty subset for bounding box:', bbox)
            continue
        subset = create_height_bucket_col(subset) 
        subset = creat_age_buckets(subset )
        # Group by specified columns and calculate both mean height and count in one go
        stats = subset.groupby(['map_simple_use', 'premise_age_bucketed', 'height_bucket'])['floor_count'] \
                .agg(mean_height='mean', count='size').reset_index()
        stats['weighted_height'] = stats['mean_height'] * stats['count']
        
        listdf.append(stats)  # Append the calculated stats to the list

    # Concatenate all DataFrames in the list into one
    full_df = pd.concat(listdf)

    # Calculate total counts and sum of weighted heights for each group across all subsets
    total_stats = full_df.groupby(['map_simple_use', 'premise_age', 'premise_floor_count']).agg(
        total_count=('count', 'sum'),
        sum_weighted_height=('weighted_height', 'sum')).reset_index()
    
    # Calculate weighted mean height for each group
    total_stats['weighted_mean'] = total_stats['sum_weighted_height'] / total_stats['total_count']

    # Cleanup before returning
    total_stats = total_stats.drop(columns=['total_count', 'sum_weighted_height'])

    total_stats = total_stats.rename(columns={'weighted_mean': 'global_average_height'})
    total_stats.to_csv('global_average_heights.csv')
    print('File saved')
    
    return total_stats

In [ ]:
# Appendix

In [29]:
medh.merge(tes_mh, on=['map_simple_use', 'premise_age', 'premise_floor_count'], how='left')

,map_simple_use,premise_age,premise_floor_count,global_average_height,height
0,Mixed Use,1870-1918,2,7.633334,7.633334
1,Mixed Use,1870-1918,4,13.800000,13.800000
2,Mixed Use,1919-1944,2,8.200000,8.200000
3,Mixed Use,1945-1959,2,8.500000,8.500000
4,Mixed Use,1960-1979,1,5.600000,5.600000
5,Mixed Use,1960-1979,2,8.533334,8.533334
6,Mixed Use,1960-1979,3,11.000000,11.000000
7,Mixed Use,1990-1999,2,9.400000,9.400000
8,Mixed Use,Pre 1837,2,9.600000,9.600000
9,Mixed Use,Pre 1837,3,10.500000,10.500000


In [28]:
tes_d = test_medheights(test , input_gpk)   
tes_mdh = tes_d.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['height'].mean()
tes_mh = tes_mdh.reset_index()

Starting  (9252.36, 6435.25920425788, 109252.36, 106435.25920425788)
Starting  (9252.36, 106435.25920425788, 109252.36, 206435.25920425786)
Empty subset
Starting  (9252.36, 206435.25920425786, 109252.36, 306435.25920425786)
Empty subset
Starting  (9252.36, 306435.25920425786, 109252.36, 406435.25920425786)
Empty subset
Starting  (9252.36, 406435.25920425786, 109252.36, 506435.25920425786)
Empty subset
Starting  (9252.36, 506435.25920425786, 109252.36, 606435.2592042579)
Empty subset
Starting  (9252.36, 606435.2592042579, 109252.36, 706435.2592042579)
Empty subset
Starting  (9252.36, 706435.2592042579, 109252.36, 806435.2592042579)
Starting  (9252.36, 806435.2592042579, 109252.36, 906435.2592042579)
Starting  (9252.36, 906435.2592042579, 109252.36, 1006435.2592042579)


In [ ]:
medh = medh.reset_index()

In [ ]:
tes_mh[tes_mh['map_simple_use']=='Residential'] 

,map_simple_use,premise_age,premise_floor_count,height
18,Residential,1837-1869,2,8.537500
19,Residential,1837-1869,4,9.200000
20,Residential,1870-1918,1,6.100000
21,Residential,1870-1918,2,8.699609
22,Residential,1870-1918,3,8.928571
23,Residential,1870-1918,4,11.300000
24,Residential,1919-1944,1,5.187500
25,Residential,1919-1944,2,8.646353
26,Residential,1919-1944,3,9.228571
27,Residential,1919-1944,4,8.916667
